In [31]:
import numpy as np
import pandas as pd
import ast

In [ ]:
df = pd.read_csv('././data/raw/movies_metadata.csv', encoding='UTF-8')

# 1. 필요한 컬럼만 남기기

df = df[['genres','imdb_id']]

# df.to_csv('./movies_processed.csv', encoding='UTF-8')

C:\Users\jinsa\AppData\Local\Temp\ipykernel_23484\3809898786.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./movies_metadata.csv', encoding='UTF-8')


In [ ]:
# 2. 결측값, 중복값, 이상치 제거

# df.info() # imdb_id에 결측값이 존재 >>> 제거

df.dropna(how='any',inplace=True)

# df.info() # 잘 제거되었음

#==========================================

# df.describe() # imdb_id에 중복값이 존재

# df[df.duplicated(subset='imdb_id')]

df = df.drop_duplicates(subset='imdb_id', keep='first', inplace=False, ignore_index=True)

#=========================================

df = df[df['genres'] != "[]"] # 빈 장르값 제거

df = df[df['imdb_id'] != "0"] # imdb_id가 0인 경우 제거

df.describe()

,genres,imdb_id
count,42979,42979
unique,4064,42979
top,"[{'id': 18, 'name': 'Drama'}]",tt0114709
freq,4996,1


In [ ]:
# 3. 장르 열 형태 변환

# df['genres'] = df['genres'].str.strip('"[]"').str.split(',')

df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)])

print(df)


                             genres    imdb_id
0       [Animation, Comedy, Family]  tt0114709
1      [Adventure, Fantasy, Family]  tt0113497
2                 [Romance, Comedy]  tt0113228
3          [Comedy, Drama, Romance]  tt0114885
4                          [Comedy]  tt0113041
...                             ...        ...
45410             [Science Fiction]  tt0112613
45411      [Drama, Action, Romance]  tt0102797
45412               [Drama, Family]  tt6209470
45413                       [Drama]  tt2028550
45414     [Action, Drama, Thriller]  tt0303758

[42979 rows x 2 columns]


In [ ]:
# 4. 장르 멀티레이블

from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

mlb = MultiLabelBinarizer()

# 변환
encoding = mlb.fit_transform(df['genres'])

# 데이터프레임화
genres_df = pd.DataFrame(encoding, columns=mlb.classes_)

# 기존 데이터와 합치기
df_final = pd.concat([df.reset_index(drop=True), genres_df], axis=1)
df_final = df_final.drop('genres', axis=1)

# print(df_final)
# print(df_final.shape)

         imdb_id  Action  Adventure  Animation  Comedy  Crime  Documentary  \
0      tt0114709       0          0          1       1      0            0   
1      tt0113497       0          1          0       0      0            0   
2      tt0113228       0          0          0       1      0            0   
3      tt0114885       0          0          0       1      0            0   
4      tt0113041       0          0          0       1      0            0   
...          ...     ...        ...        ...     ...    ...          ...   
42974  tt0112613       0          0          0       0      0            0   
42975  tt0102797       1          0          0       0      0            0   
42976  tt6209470       0          0          0       0      0            0   
42977  tt2028550       0          0          0       0      0            0   
42978  tt0303758       1          0          0       0      0            0   

       Drama  Family  Fantasy  ...  History  Horror  Music  Mys

In [ ]:
# 5. 파일저장

df_final.to_csv('././data/raw/movies_processing_done.csv', encoding='UTF-8')